<a href="https://colab.research.google.com/github/jerryw18/job_hack/blob/main/JW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install uv
!pip install python-dotenv
!uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 67.2 MB/s eta 0:00:00


In [6]:
import os
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

In [7]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

In [7]:
import phoenix as px
session = px.launch_app()

AttributeError: module 'pyarrow.lib' has no attribute 'Decimal32Type'

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [28]:
#%ls /content/job_hack/dataset/user0

from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive
fatal: destination path 'job_hack' already exists and is not an empty directory.


In [42]:

try:
    storage_context = StorageContext.from_defaults(
        persist_dir="/content/drive/MyDrive/job_hack/storage/user0"
    )
    user_docs_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False


In [40]:
# load data
if not index_loaded:
  user0_docs = SimpleDirectoryReader(
      input_files=["/content/drive/MyDrive/job_hack/dataset/user0/position0.json", "/content/drive/MyDrive/job_hack/dataset/user0/user_profile.json", "/content/drive/MyDrive/job_hack/dataset/user0/interview0.json"]
      ).load_data()

  # build index
  user_docs_index = VectorStoreIndex.from_documents(user0_docs, show_progress=True)

  # persist index, use username for storage? TODO: get username
  user_docs_index.storage_context.persist(persist_dir="/content/drive/MyDrive/job_hack/storage/user0")


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [41]:

%ls /content/drive/MyDrive/job_hack/storage/user0

default__vector_store.json  graph_store.json          index_store.json
docstore.json               image__vector_store.json


In [43]:
user0_engine = user_docs_index.as_query_engine(similarity_top_k=3, llm=llm)

In [44]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=user0_engine,
        metadata=ToolMetadata(
            name="user0_cover_letter",
            description=(
                "build a cover letter from user profile and job position."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=user0_engine,
        metadata=ToolMetadata(
            name="user0_interview_assessment",
            description=(
                "provide an assessment of how well the candidate interviewed."
            ),
        ),
    ),
]

In [45]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [46]:
response = agent.chat("Could you build a cover letter that presents a strong case on why the applicant is a good candidate for the position? Finish your chain-of-thought in at most 3 steps.")
print(str(response))

> Running step 80762371-b2e3-4474-9c6f-aa8cabacdb40. Step input: Could you build a cover letter that presents a strong case on why the applicant is a good candidate for the position? Finish your chain-of-thought in at most 3 steps.
Thought: The user is asking for a cover letter that presents a strong case for the applicant. I can use the 'user0_cover_letter' tool to generate this cover letter.
Action: user0_cover_letter
Action Input: {'input': "Applicant's profile and job position"}
Observation: The applicant, Jane Doe, is a seasoned software engineer with a Master of Science degree in Computer Science from the University of Technology. She has worked in various roles, including as a Software Engineer at Big Tech Inc. and Startup One, and currently as a Senior Software Engineer at Startup Two. Her responsibilities have ranged from developing and maintaining web applications to leading the development of a new product and mentoring junior developers. Jane is skilled in Python, JavaScrip

In [47]:
response = agent.chat("Could you provide a detailed assessment of how well the candidate interviewed based on the interview notes, the candidate's profile, and the job position the candidate applied for? Please provide feedback for each interview question and answer. Please highlight areas the candidate did well, areas the candidate could improve on, and what concrete actions the cadidate can take to better prepare for future interviews. Please finish your chain-of-thought in at most 5 steps.")
print(str(response))

> Running step 4f5a214f-30c3-4f71-9f18-92000d0f9f4c. Step input: Could you provide a detailed assessment of how well the candidate interviewed based on the interview notes, the candidate's profile, and the job position the candidate applied for? Please provide feedback for each interview question and answer. Please highlight areas the candidate did well, areas the candidate could improve on, and what concrete actions the cadidate can take to better prepare for future interviews. Please finish your chain-of-thought in at most 5 steps.
Thought: The current task requires an assessment of the candidate's interview performance. I need to use the user0_interview_assessment tool to help me answer the question.
Action: user0_interview_assessment
Action Input: {'input': "Interview notes, candidate's profile, and job position details"}
Observation: The interview took place on October 15, 2023, at Tech Innovators Inc. at 10:00 AM. The interviewers were Alice Johnson and Bob Smith. The candidate, 